In [1]:
import torchvision 
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms   
import torch.optim as optim
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.utils.prune as prune
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable


In [2]:
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
transform1 = transforms.Compose(
    [    
    transforms.RandomCrop(32, 4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandAugment(num_ops=2, magnitude=12),
    transforms.ColorJitter(0.3, 0.3, 0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261)),   
    ])
trainset = torchvision.datasets.CIFAR10(root='./', train=True,download=True,transform = transform1)
indices = list(range(0, 46000,1))
trainset1 = torch.utils.data.Subset(trainset, indices)
modelloader = torch.utils.data.DataLoader(trainset1, shuffle=True, num_workers=2, batch_size = 5000)
validset = torchvision.datasets.CIFAR10(root='./', train=True,download=True, transform = transform) 
valid = list(range(46000, 50000,1))
validset1 = torch.utils.data.Subset(validset, valid)
validloader = torch.utils.data.DataLoader(validset1, shuffle=True, num_workers=2,batch_size = 100)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 1)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable


def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                             F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)
      #  self.dropout = nn.Dropout(0.2)
        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
     #   out = self.dropout(out)
        out = self.linear(out)
        return out

def resnetlowest(classes):
    return ResNet(BasicBlock, [1, 1, 1], classes)
def resnetlower(classes):
    return ResNet(BasicBlock, [2, 2, 2], classes)
def resnet20(classes):
    return ResNet(BasicBlock, [3, 3, 3], classes)
def resnet32(classes):
    return ResNet(BasicBlock, [5, 5, 5], classes)



In [5]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 =nn.Conv2d(64, 128, 3)
        self.fc1 = nn.Linear(128 * 2 * 2, 84)
        self.fc2 = nn.Linear(84,60)
        self.fc3 = nn.Linear(60, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
      #  print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()
#model.load_state_dict(torch.load("binaryCNN"))
model.to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr = 0.001)
print(get_n_params(model))

141562


In [ ]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x


def ConvMixer(dim, depth, kernel_size=5, patch_size=2, n_classes=10):
    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for i in range(depth)],
        nn.AdaptiveAvgPool2d((1,1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [ ]:
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [5]:
def layer1transform(array,size):
    newarr = array.detach().clone()
    for i in range(size):
        if array[i] <= 5:
            newarr[i] = 0
        else: 
            newarr[i] = 1
    return newarr
def optimizedtransform(array,size):
    newarr = array.detach().clone()
    for i in range(size):
        if array[i] < 2 or array[i] > 7:
            newarr[i] = 0
        else: 
            newarr[i] = 1
   # print(newarr.size())
    return newarr
def vehicletransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] < 2 or array[i] >= 8:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 0:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 1:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 8: 
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 9: 
                newarr = torch.cat((newarr,otensor),dim=0)
               # print(newarr)
    return newdata, newarr, count
def airshiptransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] == 0 or array[i] == 8:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 0:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 8: 
                newarr = torch.cat((newarr,otensor),dim=0)
               # print(newarr)
    return newdata, newarr, count
def cartrucktransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] == 1 or array[i] == 9:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
           # print(array[i])
            if array[i] == 1:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 9: 
                newarr = torch.cat((newarr,otensor),dim=0)
               # print(newarr)
    return newdata, newarr, count
def animaltransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        if array[i] >=2 and array[i] < 8:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 2:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 3:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 4: 
                newarr = torch.cat((newarr,ttensor),dim=0)
            elif array[i] == 5: 
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 6: 
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 7: 
                newarr = torch.cat((newarr,ttensor),dim=0)
    #print(newdata.size())
   # print(newarr.size())
    return newdata, newarr, count
def catdogtransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        if array[i] == 3 or array[i] ==5:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 3:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 5: 
                newarr = torch.cat((newarr,otensor),dim=0)
               # print(newarr)
    return newdata, newarr, count


In [5]:

def vehicletransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        thtensor = torch.tensor([3])
        thtensor = thtensor.to(device)
        if array[i] < 2 or array[i] >= 8:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 0:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 1:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 8: 
                newarr = torch.cat((newarr,ttensor),dim=0)
            elif array[i] == 9: 
                newarr = torch.cat((newarr,thtensor),dim=0)
               # print(newarr)
    return newdata, newarr, count
    
def animaltransform(data, array,size):
    newdata = torch.Tensor()
    newdata = newdata.to(device)
    newarr = torch.Tensor()
    newarr = newarr.to(device)
    count=0
    for i in range(size):
        ztensor = torch.zeros(1)
        ztensor = ztensor.to(device)
        otensor = torch.Tensor([1])
        otensor = otensor.to(device)
        ttensor = torch.Tensor([2])
        ttensor = ttensor.to(device)
        thtensor = torch.Tensor([3])
        thtensor = thtensor.to(device)
        fotensor = torch.Tensor([4])
        fotensor = fotensor.to(device)
        ftensor = torch.Tensor([5])
        ftensor = ftensor.to(device)
        if array[i] >=2 and array[i] < 8:
            newdata = torch.cat((newdata,data[i].unsqueeze(0)),dim=0)       
            count = count+1
            if array[i] == 2:
                newarr = torch.cat((newarr,ztensor),dim=0)
            elif array[i] == 3:
                newarr = torch.cat((newarr,otensor),dim=0)
            elif array[i] == 4: 
                newarr = torch.cat((newarr,ttensor),dim=0)
            elif array[i] == 5: 
                newarr = torch.cat((newarr,thtensor),dim=0)
            elif array[i] == 6: 
                newarr = torch.cat((newarr,fotensor),dim=0)
            elif array[i] == 7: 
                newarr = torch.cat((newarr,ftensor),dim=0)
    #print(newdata.size())
   # print(newarr.size())
    return newdata, newarr, count

In [9]:
#model = resnetlowest(2)
model = resnet20(6)
#model = LeNet5(2)
#model = ConvMixer(256, 8, patch_size=2, kernel_size=5, n_classes=2)
model.to(device)
#model.load_state_dict(torch.load("animal5000batch"))
#model.to(device)

print(get_n_params(model))
criterion = nn.CrossEntropyLoss() 
# = optim.SGD(model.parameters(), lr=0.01, weight_decay = 0.001, momentum = 0.9)
optimizer = optim.AdamW(model.parameters(), lr=0.02, weight_decay=0.01)
#optim.Adam(model.parameters(), lr = 0.001)
#optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)

    
    

269462


In [7]:
from numpy.linalg import norm
train_losslist = []
train_losslist1 = []
#number of epochs to train the model
n_epochs = [*range(300)] # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss
scaler = torch.cuda.amp.GradScaler()
for epoch in n_epochs:
    # keep track of training and validation loss
    train_loss = 0.0
    train_loss1 = 0.0
    valid_loss = 0.0   
    ###################
    # train the model #
    ###################
    model.train()
    #model1.train()
    class_correct1 = list(0. for i in range(10))
    class_total1 = list(0. for i in range(10))
    for data, target in modelloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        newtarget = optimizedtransform(target,500)
        #print(newtarget)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        #optimizer1.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        #output1 = model1(data)
        # calculate the batch loss
        loss = criterion(output, newtarget)
        #loss.backward()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

       # optimizer.step()
        #optimizer1.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        #train_loss1 += loss1.item()*data.size(0)
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(newtarget.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(500):
            label = newtarget.data[i]
            class_correct1[label] += correct[i].item()
            class_total1[label] += 1
    ######################    
    # validate the model #
    ######################
    model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for data, target in validloader:
        # move tensors to GPU if CUDA is available
        #if train_on_gpu:
        data, target = data.cuda(), target.cuda()
        newtarget = optimizedtransform(target,100)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, newtarget)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(newtarget.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(100):
            label = newtarget.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            # calculate average losses
    valid_loss = valid_loss/len(validloader.dataset)
    train_loss = train_loss/len(modelloader.dataset)
    train_losslist.append(train_loss)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct1) / np.sum(class_total1),
    np.sum(class_correct1), np.sum(class_total1)))
    print('Validation Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
        
           

NameError: name 'optimizedtransform' is not defined

In [10]:
"""
SUBCLASS TRAINING
"""
from numpy.linalg import norm
train_losslist = []
train_losslist1 = []
#number of epochs to train the model
n_epochs = [*range(300)] # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss
scaler = torch.cuda.amp.GradScaler()
for epoch in n_epochs:
    # keep track of training and validation loss
    train_loss = 0.0
    train_loss1 = 0.0
    valid_loss = 0.0   
    ###################
    # train the model #
    ###################
    model.train()
    #model1.train()
    class_correct1 = list(0. for i in range(10))
    class_total1 = list(0. for i in range(10))
    for data, target in modelloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        temp = list(target.size())
        #print(list(target.size()))
        newdata,newtarget,size = animaltransform(data,target,temp[0])
        #lprint(newtarget)
        newtarget = newtarget.type(torch.LongTensor) 
        newtarget = newtarget.to(device)
       # print(newtarget.size())
        #print(newtarget)
        #print(newtarget)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        #optimizer1.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(newdata)
        #output1 = model1(data)
        # calculate the batch loss
        loss = criterion(output, newtarget)
        loss.backward()
        """
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        """

        optimizer.step()
        #optimizer1.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        #train_loss1 += loss1.item()*data.size(0)
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(newtarget.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(size):
            label = newtarget.data[i]
            class_correct1[label] += correct[i].item()
            class_total1[label] += 1
    ######################    
    # validate the model #
    ######################
    model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for data, target in validloader:
        # move tensors to GPU if CUDA is available
        #if train_on_gpu:
        data, target = data.cuda(), target.cuda()
        newdata,newtarget,size = animaltransform(data,target,100)
        newtarget = newtarget.type(torch.LongTensor) 
        newtarget = newtarget.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(newdata)
        # calculate the batch loss
        loss = criterion(output, newtarget)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(newtarget.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(size):
            label = newtarget.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            # calculate average losses
    valid_loss = valid_loss/len(validloader.dataset)
    train_loss = train_loss/len(modelloader.dataset)
    train_losslist.append(train_loss)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct1) / np.sum(class_total1),
    np.sum(class_correct1), np.sum(class_total1)))
    print('Validation Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
        

Epoch: 0 	Training Loss: 2.615837 	Validation Loss: 6844.218896
Training Accuracy (Overall): 16% (4697/27632)
Validation Accuracy (Overall): 16% (388/2368)
Epoch: 1 	Training Loss: 1.817883 	Validation Loss: 17.174475
Training Accuracy (Overall): 19% (5328/27632)
Validation Accuracy (Overall): 16% (384/2368)
Epoch: 2 	Training Loss: 1.773839 	Validation Loss: 1.839972
Training Accuracy (Overall): 22% (6122/27632)
Validation Accuracy (Overall): 23% (563/2368)
Epoch: 3 	Training Loss: 1.740445 	Validation Loss: 1.740845
Training Accuracy (Overall): 24% (6849/27632)
Validation Accuracy (Overall): 23% (566/2368)
Epoch: 4 	Training Loss: 1.718048 	Validation Loss: 1.678947
Training Accuracy (Overall): 26% (7190/27632)
Validation Accuracy (Overall): 28% (670/2368)
Epoch: 5 	Training Loss: 1.689913 	Validation Loss: 1.678343
Training Accuracy (Overall): 28% (7858/27632)
Validation Accuracy (Overall): 28% (678/2368)
Epoch: 6 	Training Loss: 1.662700 	Validation Loss: 1.610586
Training Accuracy

In [12]:
torch.save(model.state_dict(), "resnetdropout5000batch")

In [16]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
model.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        initialtarget = optimizedtransform(target,100)
        initialtarget = initialtarget.type(torch.LongTensor) 
        initialtarget = initialtarget.to(device)
        #print(initialtarget.size())
      #  print(initialtarget)
        output1 = model(data)
        _, pred = torch.max(output1, 1) 
        correct_tensor = pred.eq(initialtarget.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      #  print(correct)
        """
        PRINT SAMPLES MISCATEGORIZED
        DECREASE MODEL SIZE
        TRAIN ANOTHER LAYER
        OPTIMIZE BINARY CLASSIFIACTION DATASET
        """
        for i in range(100):
            label = initialtarget.data[i]
          #  print(label)
            class_correct[label] += correct[i]
            class_total[label] += 1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
               np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 95% (3821/4000)
Test Accuracy of automobile: 97% (5856/6000)
Test Accuracy of  bird: N/A (no training examples)
Test Accuracy of   cat: N/A (no training examples)
Test Accuracy of  deer: N/A (no training examples)
Test Accuracy of   dog: N/A (no training examples)
Test Accuracy of  frog: N/A (no training examples)
Test Accuracy of horse: N/A (no training examples)
Test Accuracy of  ship: N/A (no training examples)
Test Accuracy of truck: N/A (no training examples)

Test Accuracy (Overall): 96% (9677/10000)


In [20]:
test_loss = 0.
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
matrix = np.zeros((10,10))
train_on_gpu = torch.cuda.is_available()
model.eval()
# iterate over test data
for data, target in testloader:
    # move tensors to GPU if CUDA is available
    data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    newdata,initialtarget,size = animaltransform(data,target,100)
    initialtarget = initialtarget.type(torch.LongTensor) 
    initialtarget = initialtarget.to(device)
        #print(initialtarget.size())
      #  print(initialtarget)
    output1 = model(newdata)
    # calculate the batch loss
    loss = criterion(output1, initialtarget)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output1, 1)    
    # compare predictions to true label
  #  print(pred)
    correct_tensor = pred.eq(initialtarget.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(size):
        label = initialtarget.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        matrix[label][pred[i]] +=1

# average test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))
matrix = matrix/10
print(classes)
print(matrix)
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.509587

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
[[85.7  1.6  6.8  2.9  2.6  0.4  0.   0.   0.   0. ]
 [ 2.4 75.4  3.3 14.8  3.6  0.5  0.   0.   0.   0. ]
 [ 0.9  1.6 94.8  1.2  0.5  1.   0.   0.   0.   0. ]
 [ 1.1  5.9  1.7 89.7  1.2  0.4  0.   0.   0.   0. ]
 [ 1.3  1.4  1.2  1.1 94.9  0.1  0.   0.   0.   0. ]
 [ 0.6  1.4  3.4  2.7  0.4 91.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]]
Test Accuracy of airplane: 85% (857/1000)
Test Accuracy of automobile: 75% (754/1000)
Test Accuracy of  bird: 94% (948/1000)
Test Accuracy of   cat: 89% (897/1000)
Test Accuracy of  deer: 94% (949/1000)
Test Accuracy of   dog: 91% (915/1000)
Test Accuracy of  frog: N/A (no training examples)
Test Accuracy of horse: N/A (no training examples)
Test Accuracy

In [9]:
model1 = resnetlowest(2)
animal = resnet20(6)
vehicle = resnetlowest(2)
cartruck = resnetlowest(2)
airship = resnetlowest(2)
model1.load_state_dict(torch.load("binarylowest"))
animal.load_state_dict(torch.load("animal5000batch"))
vehicle.load_state_dict(torch.load("vehiclelowesttree"))
cartruck.load_state_dict(torch.load("cartrucktree"))
airship.load_state_dict(torch.load("airshiptree"))
cartruck.to(device)
airship.to(device)
model1.to(device)
animal.to(device)
vehicle.to(device)


ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=

In [27]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
matrix = np.zeros((10,10))
model1.eval()
animal.eval()
vehicle.eval()
cartruck.eval()
airship.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        output1 = model1(data)
        _, pred = torch.max(output1, 1) 
        if pred == 0:
            output2 = vehicle(data)
            _, pred2 = torch.max(output2, 1)
            if pred2 == 0:
                output3 = airship(data)
                _, pred3 = torch.max(output3, 1)
               # print(pred3)
                predfinal = pred3
                if pred3[0] != 0:
                    predfinal = pred3 + 7
            if pred2 == 1:
                output3 = cartruck(data)
                _, pred3 = torch.max(output3, 1)
                predfinal = pred3
              #  print(pred3)
               # print(target)
                if pred3 == 0:
                    predfinal =predfinal + 1
                else:                    
                    predfinal =predfinal + 8   
        if pred == 1:
            output2 = animal(data)
            _, pred2 = torch.max(output2, 1)
            predfinal = pred2 +2
        correct_tensor = predfinal.eq(target.data.view_as(predfinal))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        label = target.data[0]
        class_correct[label] += correct.item()
        class_total[label] += 1
        matrix[label][predfinal] +=1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 85% (855/1000)
Test Accuracy of automobile: 87% (873/1000)
Test Accuracy of  bird: 80% (808/1000)
Test Accuracy of   cat: 73% (735/1000)
Test Accuracy of  deer: 93% (937/1000)
Test Accuracy of   dog: 88% (887/1000)
Test Accuracy of  frog: 94% (942/1000)
Test Accuracy of horse: 89% (895/1000)
Test Accuracy of  ship: 92% (921/1000)
Test Accuracy of truck: 89% (894/1000)

Test Accuracy (Overall): 87% (8747/10000)


In [24]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
model1.eval()
animal.eval()
vehicle.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        output1 = model1(data)
        _, pred = torch.max(output1, 1) 
        if pred == 0:
            output2 = vehicle(data)
            _, pred2 = torch.max(output2, 1)
            predfinal = pred2
            if predfinal > 1:
                predfinal = predfinal + 6
        if pred == 1:
            output2 = animal(data)
            _, pred2 = torch.max(output2, 1)
            predfinal = pred2 + 2
        correct_tensor = predfinal.eq(target.data.view_as(predfinal))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        label = target.data[0]
        class_correct[label] += correct.item()
        class_total[label] += 1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 87% (876/1000)
Test Accuracy of automobile: 90% (904/1000)
Test Accuracy of  bird: 80% (808/1000)
Test Accuracy of   cat: 73% (735/1000)
Test Accuracy of  deer: 93% (937/1000)
Test Accuracy of   dog: 88% (887/1000)
Test Accuracy of  frog: 94% (942/1000)
Test Accuracy of horse: 89% (895/1000)
Test Accuracy of  ship: 91% (918/1000)
Test Accuracy of truck: 95% (952/1000)

Test Accuracy (Overall): 88% (8854/10000)


In [11]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
train_on_gpu = torch.cuda.is_available()
matrix = np.zeros((10,10))
vehicle.eval()
cartruck.eval()
airship.eval()
for data, target in testloader:
        data, target = data.to(device), target.to(device)
        if target <= 1 or target > 7:
            output2 = vehicle(data)
            _, pred2 = torch.max(output2, 1)
            if pred2 == 0:
                output3 = airship(data)
                _, pred3 = torch.max(output3, 1)
               # print(pred3)
                predfinal = pred3
                if pred3[0] != 0:
                    predfinal = pred3 + 7
            if pred2 == 1:
                output3 = cartruck(data)
                _, pred3 = torch.max(output3, 1)
                predfinal = pred3
              #  print(pred3)
               # print(target)
                if pred3 == 0:
                    predfinal =predfinal + 1
                else:                    
                    predfinal =predfinal + 8   
            correct_tensor = predfinal.eq(target.data.view_as(predfinal))
            correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
            label = target.data[0]
            class_correct[label] += correct.item()
            class_total[label] += 1
            matrix[label][predfinal] +=1
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of airplane: 93% (938/1000)
Test Accuracy of automobile: 89% (895/1000)
Test Accuracy of  bird: N/A (no training examples)
Test Accuracy of   cat: N/A (no training examples)
Test Accuracy of  deer: N/A (no training examples)
Test Accuracy of   dog: N/A (no training examples)
Test Accuracy of  frog: N/A (no training examples)
Test Accuracy of horse: N/A (no training examples)
Test Accuracy of  ship: 94% (948/1000)
Test Accuracy of truck: 90% (909/1000)

Test Accuracy (Overall): 92% (3690/4000)
